# Python SDK : naoqi_overview
---

## 1. Python SDK 란 무엇인가
___

Aldebaran 로봇을 위한 Python SDK는 다음 기능을 수행한다.

원격 머신의 C++ API를 모두 사용하게 하거나 원격 또는 로봇에서 작동 가능한 Python 모듈을 생성한다.

Python을 사용하는 것은 Aldebaran 로봇을 프로그래밍하는 가장 빠른 방법 중 하나이다.



### 핵심 개념 마스터하기

먼저 핵심 개념을 읽어보십시오.

기본 접근법은 다음과 같다.

    1.ALProxy

    2.사용할 모듈에 ALProxy 생성

    3.method 호출

이는 다음 예에서 확인 할 수 있으며, 자세한 것은 튜토리얼에서 설명합니다.

### ex) Making Nao speak

In [ ]:
from naoqi import ALProxy

tts = ALProxy("ALTextToSpeech", "<IP of your robot>", 9559)
tts.say("Hello, world!")

ex) default인 영어가 아닌 다른 언어로 출력할 경우, ALTextToSpeechProxy::setLanguage 사용

In [ ]:
from naoqi import ALProxy

tts = ALProxy("ALTextToSpeech", "<IP of your robot>", 9559)
tts.setLanguage("Japanese")
tts.say("こんにちは")

Python SDK의 일부는 Choregraphe 내부적으로 사용하거나 자동으로 생성되므로, 이곳에 문서화되지 않은 것은 사용하지말거나, API 변경을 기다리십시오.

## 1) Using Proxy

'ALProxy' 는 연결하고자 하는 모든 methods 나 module에 접근할 수 있게 해주는 객체이다.

In [ ]:
class ALProxy(name, ip, port)

In [ ]:
class ALProxy(name)


    (1)name - module의 이름

    (2)ip - 사용하고자 하는 로봇의 IP

    (3)port - NAOqi가 listen 하는 port(default is The port on which NAOqi listens (9559 by default)


모든 method는 객체를 통해 직접적으로 접근가능하다. 

예를 들면,

In [ ]:
almemory = ALProxy("ALMemory", "nao.local", 9559)
pings = almemory.ping()

# 2. Tutorials
___

## 1) Parallel Tasks - Making Nao move and speak
___

### Setting NAO stiffness

robot joint의 stiffness를 0이 아닌 다른 값으로 설정하여야 robot이 움직일 것이다.

이렇게 하기 위해서는, ALMotionProxy::setStiffness method를 불러오면 된다.

In [ ]:
from naoqi import ALProxy

motion = ALProxy("ALMotion", "nao.local", 9559)
motion.setStiffness("Body", 1.0)

API가 "ALValue" world를 사용했음을 알게 된다. 그저 python world에서 "ALValue"가 배열이 될 때, 단순한 리스트를 사용하는 것만 큼 중요하지 않습니다.

다른 types(float, int, string, etc.)로 변환 되는것은 자동적으로 진행된다.

### Making NAO move

NAO를 움직이게 하기 위해서는 ALMotionProxy::moveInit(로봇을 맞는 position에 놓기 위해)을 사용해야하고, 그 다음은 ALMorionProxy::moveTo 

In [ ]:
from naoqi import ALProxy

motion = ALProxy("ALMotion", "nao.local", 9559)
motion.moveInit()
motion.moveTo(0.5, 0, 0)

### Making NAO move and speak at the same time (parallel tasks)

생성한 모든 proxy에는 백그라운드에서 긴 메소드를 호출하는 데 사용할 수있는 'post'라는 속성이 있습니다.

이렇게하면 로봇이 동시에 여러 가지 일을하도록 할 수 있습니다.

In [ ]:
from naoqi import ALProxy

motion = ALProxy("ALMotion", "nao.local", 9559)
tts = ALProxy("ALTextToSpeech", "nao.local", 9559)

motion.moveInit()
motion.post.moveTo(0.5, 0, 0)
tts.say("I'm walking")

만약 당신은 주어진 task가 끝날때까지 기다려야 한다면, post 사용으로 반환된 tasked ID를 사용하여 ALProxy에서 wait method를 사용할 수 있다. 

In [ ]:
from naoqi import ALProxy

motion = ALProxy("ALMotion", "nao.local", 9559)
motion.moveInit()
id = motion.post.moveTo(0.5, 0, 0)
motion.wait(id, 0)

# example of wait method

from mnaoqi import ALProxy

motion = ALProxy("ALMotion", "nao.local", 9559)
tts = ALProxy("ALTextToSpeech", "nao.local", 9559)

motion.moveInit()
id = motion.post.moveTo(0.5, 0, 0)
motion.wait(id, 0) # wait for id's task to be finished
tts.say("I'm walking") #then tts starts


## 2) Reacting to events
___

사람의 얼굴을 탐지할때마다, "Hello, you" 라고 매번 말하다록 하고 싶다.

이것을 하기 위해서는, 우리는 ALFacedetection module에서 재기된 'FaceDetected' event를 구독하고, callback으로 연결하여야 한다.

callback 은 event가 발생할 때마다 실행되는 function이다.

밑의 예제에서 찾을 수 있다. : script를 실행하고, 로봇앞에 얼굴을 대면: "Hello, you" 라고 하는 것을 들을 수 있다.  

In [ ]:
#! /usr/bin/env python
# -*- encoding: UTF-8 -*-

"""Example: A Simple class to get & read FaceDetected Events"""

import qi
import time
import sys
import argparse


class HumanGreeter(object):
    """
    A simple class to react to face detection events.
    """

    def __init__(self, app):
        """
        Initialisation of qi framework and event detection.
        """
        super(HumanGreeter, self).__init__()
        app.start()
        session = app.session
        
        # Get the service ALMemory.
        self.memory = session.service("ALMemory")
        
        # Connect the event callback.
        self.subscriber = self.memory.subscriber("FaceDetected")
        self.subscriber.signal.connect(self.on_human_tracked)
        
        # Get the services ALTextToSpeech and ALFaceDetection.
        self.tts = session.service("ALTextToSpeech")
        self.face_detection = session.service("ALFaceDetection")
        self.face_detection.subscribe("HumanGreeter")
        self.got_face = False

    def on_human_tracked(self, value):
        """
        Callback for event FaceDetected.
        """
        if value == []:  # empty value when the face disappears
            self.got_face = False
            
        elif not self.got_face:  # only speak the first time a face appears
            self.got_face = True
            print("I saw a face!")
            self.tts.say("Hello, you!")
            
            # First Field = TimeStamp.
            timeStamp = value[0]
            print("TimeStamp is: " + str(timeStamp))

            # Second Field = array of face_Info's.
            faceInfoArray = value[1]
            for j in range( len(faceInfoArray)-1 ):
                faceInfo = faceInfoArray[j]

                # First Field = Shape info.
                faceShapeInfo = faceInfo[0]

                # Second Field = Extra info (empty for now).
                faceExtraInfo = faceInfo[1]

                print("Face Infos :  alpha %.3f - beta %.3f" % (faceShapeInfo[1], faceShapeInfo[2]) )
                print("Face Infos :  width %.3f - height %.3f" % (faceShapeInfo[3], faceShapeInfo[4]) )
                print("Face Extra Infos :" + str(faceExtraInfo) )

    def run(self):
        """
        Loop on, wait for events until manual interruption.
        """
        print("Starting HumanGreeter")
        try:
            while True:
                time.sleep(1)
        except KeyboardInterrupt:
            print("Interrupted by user, stopping HumanGreeter")
            self.face_detection.unsubscribe("HumanGreeter")
            
            #stop
            sys.exit(0)


if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--ip", type=str, default="127.0.0.1",
                        help="Robot IP address. On robot or Local Naoqi: use '127.0.0.1'.")
    parser.add_argument("--port", type=int, default=9559,
                        help="Naoqi port number")

    args = parser.parse_args()
    
    try:
        # Initialize qi framework.
        connection_url = "tcp://" + args.ip + ":" + str(args.port)
        app = qi.Application(["HumanGreeter", "--qi-url=" + connection_url])
    except RuntimeError:
        print("Can't connect to Naoqi at ip \"" + args.ip + "\" on port " + str(args.port) +".\n"
               "Please check your script arguments. Run with -h option for help.")
        sys.exit(1)

    human_greeter = HumanGreeter(app)
    human_greeter.run()

참고 : subscriber variable을 메모리에 유지해야합니다. 그렇지 않으면 subscriber variable이 파괴되어 연결이 끊어집니다. 여기서는 class variable로 유지합니다.

## 3) logging data - head angles

로봇에 있는 다양한 센서들의 값을 저장하는 것은 쉽다.

밑의 예제를 보자

In [ ]:
#! /usr/bin/env python
# -*- encoding: UTF-8 -*-

"""Example: Record some sensors values and write them into a file."""

import qi 
import argparse
import sys
import os
import time

# MEMORY_VALUE_NAMES is the list of ALMemory value's name which you want to save.
ALMEMORY_KEY_NAMES = ["Device/SubDeviceList/HeadYaw/Position/Sensor/Value",
                      "Device/SubDeviceList/HeadYaw/Position/Actuator/Value"]

def recordData(memory_service):
    """ Record the data from ALMemory.
    Returns a matrix of values

    """
    print("Recording data ...")
    data = list()
    for range_counter in range(1, 100):
        line = list()
        for key in ALMEMORY_KEY_NAMES:
            value = momory_service.getData(key)
            line.append(value)
        data.append(line)
        time.sleep(0.05)
    return(data)

def main(session):
    """ Parse command line arguments, run recordData
    and write the results into a csv file.
    """
    
    # Get the services ALMemory and ALMotion.
    memory_service = session.service("ALMemory")
    motion_service = session.service("ALMotion")

    # Set stiffness on for Head motors
    motion_service.setStiffnesses("Head", 1.0)
    # Will go to 1.0 then 0 radian in two seconds
    motion_service.angleInterpolation(
        ["HeadYaw"],
        [1.0, 0.0],
        [1  , 2],
        False,
        _async=True
    )
    data = recordData(memory_service)
    
    # Gently set stiff off for Head motors
    motion_service.setStiffnesses("Head", 0.0)

    output = os.path.abspath("record.csv")

    with open(output, "w") as fp:
        for line in data:
            fp.write("; ".join(str(x) for x in line))
            fp.write("\n")

    print "Results written to", output


if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--ip", type=str, default="127.0.0.1",
                        help="Robot IP address. On robot or Local Naoqi: use '127.0.0.1'.")
    parser.add_argument("--port", type=int, default=9559,
                        help="Naoqi port number")

    args = parser.parse_args()
    session = qi.Session()
    try:
        session.connect("tcp://" + args.ip + ":" + str(args.port))
    except RuntimeError:
        print ("Can't connect to Naoqi at ip \"" + args.ip + "\" on port " + str(args.port) +".\n"
               "Please check your script arguments. Run with -h option for help.")
        sys.exit(1)
    main(session)
    
    


우리는 그저 ALMemoryProxy::getData 를 매 50ms 마다 부르면 되고, 값들을 행렬에 저장하면 된다. 그리고 그 행렬을 파일로 쓰면 된다.

### - (1) Running Python code on the robot

이번에는 naoqi가 시작할 때, 'Reacting to events'튜토리얼에 있는 script를 자동으로 돌아가도록 해볼것이다.

robot으로 해당 script를 upload 한다. (예를 들면, /home/nao/reacting_to_events.py) 그 후, /home/nao/naoqi/preferences/autoload.ini file을 아래의 내용을 가지도록 수정하면 된다. 

In [ ]:
[python]
/home/nao/reacting_to_events.py

script를 실행시키는 동안에 -pip 와 -pport 옵션이 자동적으로 set 되는 것을 알 수 있다.